In [1]:
import numpy as np
from math import floor, ceil
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score, mean_squared_error, classification_report, roc_auc_score
from keras.utils import np_utils
from dependencies import NeuralNetwork, FCLayer, ActivationLayer, tanh, tanh_derivative, sigmoid, sigmoid_derivative, relu, relu_derivative, identity, identity_derivative, mse, mse_derivative, cross_entropy_binary, cross_entropy_binary_derivative, NetworkGenerator
from ray import tune
from ray.tune.schedulers import AsyncHyperBandScheduler
import pickle

In [2]:
hyperparameters = {
    "type": "Regression",
    #"number_of_layers": [3, 4, 5],
    #"hidden_layer_sizes": [2, 1, 1, 1, 1],
    "output_layer_size": 1,
    #"activation_function": "sigmoid",
    #"learning_rate": 0.1,
    #"epochs": 100,
    #"batch_size": 1,
    #"loss_function": "mean_squared_error",
    #"optimizer": "adam",
}

In [2]:
X, Y = load_wine(return_X_y=True)
print(X.shape, Y.shape)
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.2)
#Normalizing data
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(178, 13) (178,)
(142, 13) (36, 13) (142, 3) (36, 3)


In [4]:
class NetworkGenerator:
    def __init__(self, X_train, y_train, X_test, y_test, type):
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test
        self.type = type
        self.input_layer_size = X_train.shape[1]
        if(self.type == "Regression"):
            self.output_layer_size = 1
            self.activation_function = relu
            self.activation_function_derivative = relu_derivative
        else: 
            self.output_layer_size = y_train.shape[1]
            self.activation_function = tanh
            self.activation_function_derivative = tanh_derivative
        self.middle_layer_size = ceil((2*(self.input_layer_size + self.output_layer_size))/3)
        
#interpolating the hidden layer sizes
    def __interpolate_hidden_layer_sizes(self, middle_layer_size, number_of_layers):
        hidden_layer_sizes = [0 for x in range(number_of_layers)]
        layers_before_middle = floor(number_of_layers/2) #2
        hidden_layer_sizes[layers_before_middle] = middle_layer_size
        index = layers_before_middle - 1
        for i in range(layers_before_middle, 0, -1):
            hidden_layer_sizes[index] = ceil(2*(self.input_layer_size + hidden_layer_sizes[i])/3)
            index -= 1
        
        index = layers_before_middle + 1
        for i in range(layers_before_middle, number_of_layers - 1, 1):
            hidden_layer_sizes[index] = ceil(2*(hidden_layer_sizes[i] + self.output_layer_size)/3)
            index += 1
        return hidden_layer_sizes

    def __create_network(self, middle_layer_size, number_of_layers, epochs, alpha, batch_size):
        network = NeuralNetwork()
        hidden_layer_sizes = self.__interpolate_hidden_layer_sizes(middle_layer_size, number_of_layers)
        hidden_layer_sizes.insert(0, self.input_layer_size)
        hidden_layer_sizes.append(self.output_layer_size)
        for i in range(len(hidden_layer_sizes) - 1):
            network.add(FCLayer(hidden_layer_sizes[i], hidden_layer_sizes[i + 1]))
            if i < len(hidden_layer_sizes) - 1:
                network.add(ActivationLayer(self.activation_function, self.activation_function_derivative))
        if self.type == "Classification":
            network.add(ActivationLayer(sigmoid, sigmoid_derivative))
            network.use(mse, mse_derivative)  #TODO: change loss function
        else: 
            network.add(ActivationLayer(identity, identity_derivative))
            network.use(mse, mse_derivative)
        network.fit(self.X_train, self.y_train, epochs=epochs, learning_rate=alpha, batch_size=batch_size)
        return network
            
    def __network_generator(self, config, reporter):
        network = self.__create_network(config["middle_layer_size"], config["number_of_layers"], config["epochs"], config["alpha"], config["batch_size"])
        y_pred = np.array(network.predict(self.X_test))
        if self.type == "Classification":
            reporter(config, mean_accuracy = accuracy_score(self.y_test, y_pred.round()), network=network)
        else:
            reporter(config, mean_loss = mean_squared_error(self.y_test, y_pred))
        
    def __train_network(self):
        scheduler = AsyncHyperBandScheduler()
        if self.type == "Classification":
            res = tune.run(
            self.__network_generator,
            name="my_exp",
            metric="mean_accuracy",
            stop={"mean_accuracy": 0.9},
            mode="max",
            scheduler=scheduler,
            config={
                "number_of_layers": tune.grid_search([6, 7, 8, 9]),
                "middle_layer_size": tune.grid_search([8, 16, 32]),#, 64, 128]),
                "epochs": tune.grid_search([10000]),#, 2000, 3000, 4000, 5000]),
                "alpha": tune.grid_search([0.001, 0.01, 0.1]),#, 1]),
                "batch_size": tune.grid_search([8, 16, 32]),#, 8, 16, 32]),
            }
            )
            results = {k: v for k, v in sorted(res.results.items(), key=lambda item: (item[1]["mean_accuracy"], -item[1]["time_this_iter_s"]), reverse=True)}
                
        else:
            res = tune.run(
            self.__network_generator,
            name="my_exp",
            metric="mean_loss",
            stop={"mean_loss": 0.2},
            mode="min",
            scheduler=scheduler,
            config={
                "number_of_layers": tune.grid_search([3, 4, 5]),
                "middle_layer_size": tune.grid_search([8, 16, 32, 64, 128]),
                "epochs": tune.grid_search([500, 1000, 2000, 3000, 4000, 5000]),
                "alpha": tune.grid_search([0.001, 0.01, 0.1, 1]),
            },
            )
            results = {k: v for k, v in sorted(res.results.items(), key=lambda item: (item[1]["mean_loss"], item[1]["time_this_iter_s"]))}
        return list(results.values())[0]
    
    def get_best_network(self):
        best_result = self.__train_network()
        print(best_result)
        accuracy = best_result["mean_accuracy"] if self.type == "Classification" else best_result["mean_loss"]
        config = best_result["config"]
        network = best_result["network"]
        #network = self.__create_network(config["middle_layer_size"], config["number_of_layers"], config["epochs"], config["alpha"])
        return network, accuracy, config
    
    def import_model(self, filepath):
        with open(filepath, 'rb') as f:
            res = pickle.load(f, encoding='bytes')
        if not isinstance(res, NeuralNetwork):
            raise TypeError('File does not exist or is corrupted')
        return res

    def export_model(self, filepath, network):
        with open(filepath, 'wb') as f:
            pickle.dump(network, f)

In [3]:
network_generator = NetworkGenerator(X_train, y_train, X_test, y_test, "Classification")
network = network_generator.get_best_network()
#sizes = network_generator.interpolate_hidden_layer_sizes(128, 5)
#print(sizes)

c:\Users\Public\anaconda3\lib\site-packages\ray\_private\services.py:238: UserWarning: Not all Ray Dashboard dependencies were found. To use the dashboard please install Ray using `pip install ray[default]`. To disable this message, set RAY_DISABLE_IMPORT_WARNING env var to '1'.
  warnings.warn(warning_message)
2022-05-26 11:45:09,981	INFO registry.py:66 -- Detected unknown callable for trainable. Converting to class.
2022-05-26 11:45:10,475	ERROR syncer.py:72 -- Log sync requires rsync to be installed.


Trial name,status,loc,alpha,epochs,middle_layer_size,number_of_layers
__network_generator_88552_00000,RUNNING,,0.001,500,8,6
__network_generator_88552_00001,PENDING,,0.01,500,8,6
__network_generator_88552_00002,PENDING,,0.1,500,8,6
__network_generator_88552_00003,PENDING,,0.001,1000,8,6
__network_generator_88552_00004,PENDING,,0.01,1000,8,6
__network_generator_88552_00005,PENDING,,0.1,1000,8,6
__network_generator_88552_00006,PENDING,,0.001,500,16,6
__network_generator_88552_00007,PENDING,,0.01,500,16,6
__network_generator_88552_00008,PENDING,,0.1,500,16,6
__network_generator_88552_00009,PENDING,,0.001,1000,16,6


Trial name,status,loc,alpha,epochs,middle_layer_size,number_of_layers
__network_generator_88552_00000,RUNNING,,0.001,500,8,6
__network_generator_88552_00001,RUNNING,,0.01,500,8,6
__network_generator_88552_00002,RUNNING,,0.1,500,8,6
__network_generator_88552_00003,RUNNING,,0.001,1000,8,6
__network_generator_88552_00004,RUNNING,,0.01,1000,8,6
__network_generator_88552_00005,RUNNING,,0.1,1000,8,6
__network_generator_88552_00006,RUNNING,,0.001,500,16,6
__network_generator_88552_00007,RUNNING,,0.01,500,16,6
__network_generator_88552_00008,PENDING,,0.1,500,16,6
__network_generator_88552_00009,PENDING,,0.001,1000,16,6


2022-05-26 11:45:18,059	WARNING worker.py:1215 -- Failed to unpickle actor class 'ImplicitFunc' for actor ID cd0ff9c899723ccda9487e3f01000000. Traceback:
Traceback (most recent call last):
  File "c:\Users\Public\anaconda3\lib\site-packages\ray\_private\function_manager.py", line 503, in _load_actor_class_from_gcs
    actor_class = pickle.loads(pickled_class)
ModuleNotFoundError: No module named 'neural_network'

2022-05-26 11:45:18,083	ERROR trial_runner.py:773 -- Trial __network_generator_88552_00003: Error processing event.
Traceback (most recent call last):
  File "c:\Users\Public\anaconda3\lib\site-packages\ray\tune\trial_runner.py", line 739, in _process_trial
    results = self.trial_executor.fetch_result(trial)
  File "c:\Users\Public\anaconda3\lib\site-packages\ray\tune\ray_trial_executor.py", line 746, in fetch_result
    result = ray.get(trial_future[0], timeout=DEFAULT_GET_TIMEOUT)
  File "c:\Users\Public\anaconda3\lib\site-packages\ray\_private\client_mode_hook.py", line 8

Result for __network_generator_88552_00003:
  {}
  


...

Trial name,status,loc,alpha,epochs,middle_layer_size,number_of_layers
__network_generator_88552_00016,RUNNING,,0.01,1000,32,6
__network_generator_88552_00017,RUNNING,,0.1,1000,32,6
__network_generator_88552_00018,RUNNING,,0.001,500,8,7
__network_generator_88552_00019,RUNNING,,0.01,500,8,7
__network_generator_88552_00020,RUNNING,,0.1,500,8,7
__network_generator_88552_00021,RUNNING,,0.001,1000,8,7
__network_generator_88552_00022,RUNNING,,0.01,1000,8,7
__network_generator_88552_00024,PENDING,,0.001,500,16,7
__network_generator_88552_00025,PENDING,,0.01,500,16,7
__network_generator_88552_00026,PENDING,,0.1,500,16,7


2022-05-26 11:45:40,109	WARNING tune.py:518 -- SIGINT received (e.g. via Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C one more time (or send SIGINT/SIGKILL/SIGTERM) to skip. 


Trial name,status,loc,alpha,epochs,middle_layer_size,number_of_layers
__network_generator_88552_00016,RUNNING,,0.01,1000,32,6
__network_generator_88552_00017,RUNNING,,0.1,1000,32,6
__network_generator_88552_00018,RUNNING,,0.001,500,8,7
__network_generator_88552_00019,RUNNING,,0.01,500,8,7
__network_generator_88552_00020,RUNNING,,0.1,500,8,7
__network_generator_88552_00021,RUNNING,,0.001,1000,8,7
__network_generator_88552_00022,RUNNING,,0.01,1000,8,7
__network_generator_88552_00023,RUNNING,,0.1,1000,8,7
__network_generator_88552_00024,PENDING,,0.001,500,16,7
__network_generator_88552_00025,PENDING,,0.01,500,16,7


2022-05-26 11:45:41,025	ERROR tune.py:557 -- Trials did not complete: [__network_generator_88552_00000, __network_generator_88552_00001, __network_generator_88552_00002, __network_generator_88552_00003, __network_generator_88552_00004, __network_generator_88552_00005, __network_generator_88552_00006, __network_generator_88552_00007, __network_generator_88552_00008, __network_generator_88552_00009, __network_generator_88552_00010, __network_generator_88552_00011, __network_generator_88552_00012, __network_generator_88552_00013, __network_generator_88552_00014, __network_generator_88552_00015, __network_generator_88552_00016, __network_generator_88552_00017, __network_generator_88552_00018, __network_generator_88552_00019, __network_generator_88552_00020, __network_generator_88552_00021, __network_generator_88552_00022, __network_generator_88552_00023, __network_generator_88552_00024, __network_generator_88552_00025, __network_generator_88552_00026, __network_generator_88552_00027, __net

KeyError: 'mean_accuracy'

In [ ]:
y_pred = np.array(network[0].predict(X_test))
print(accuracy_score(y_test, y_pred.round()), network[1], network[2])

In [ ]:
import pandas as pd
training_data = pd.read_csv('suggestions.csv')

for col in ['Age', 'Fare']:
    normalized = normalize([np.asarray(training_data[col])])
    training_data.drop(col, axis=1, inplace=True)
    training_data[col] = normalized[0]
        
X_train, X_test, y_train, y_test = train_test_split(
    training_data.loc[:, training_data.columns != 'Survived'], training_data["Survived"], test_size=0.20)

X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
network_generator = NetworkGenerator(X_train, y_train, X_test, y_test, "Classification")
network = network_generator.get_best_network()

In [ ]:
y_pred = np.array(network[0].predict(X_test))
print(accuracy_score(y_test, y_pred.round()), network[1], network[2])
print(classification_report(y_test, y_pred.round(), zero_division=1))
roc_auc_score(y_test, y_pred)